In [ ]:
    df = pd.read_csv(uri)
    df2 = df[df['DEPARTAMENTO']=='LORETO']
    df2 = df2[["DEPARTAMENTO","DISTRITO","CENTRO POBLADO", "LATITUD", "LONGITUD"]]
    df2["CP"] = df2.index
    dftemp=df2["CP"]
    dftry=pd.DataFrame(data=dftemp)
    import itertools
    cps=df2['CP']
    list(itertools.permutations(cps,2))
    centros=df2['CENTRO POBLADO']
    list(itertools.permutations(centros,2))
    latitudes=df2['LATITUD']
    list(itertools.permutations(latitudes,2))
    longitudes=df2['LONGITUD']
    list(itertools.permutations(latitudes,2))

    combinacionesCentros = list(itertools.permutations(centros,2))
    combinacionesLatitudes = list(itertools.permutations(latitudes,2))
    combinacionesLongitudes = list(itertools.permutations(longitudes,2))
    combinacionesCps = list(itertools.permutations(cps,2))
    df3 = pd.DataFrame(combinacionesCentros,columns=['Origen','Destino'])
    df4 = pd.DataFrame(combinacionesLatitudes,columns=['LatitudO','LatitudD'])
    df4 = df4.apply(pd.to_numeric)
    df5 = pd.DataFrame(combinacionesLongitudes,columns=['LongitudO','LongitudD'])
    df5 = df5.apply(pd.to_numeric)
    dfx = pd.DataFrame(combinacionesCps,columns=['CPO','CPD'])
    dfx = dfx.apply(pd.to_numeric)
    df6=df3.join(df4)
    df6.join(df5)
    df7=df6.join(df5)
    dfz=df7.join(dfx)

    dfz["Distancia"] = dfz.apply(haversine,axis=1)
    df8=dfz[["Origen","Destino","Distancia","CPD","LatitudD","LongitudD"]]
    df9=df8.assign(IdDP=lambda x: x.CPD * 0 + 1)
    DEPARTAMENTO=df9.to_dict('index')

    dictionary = {}
    for i in DEPARTAMENTO:
        a = DEPARTAMENTO[i]['Origen']
        dictionary[a] =[]
    for i in DEPARTAMENTO:
        a = DEPARTAMENTO[i]['Origen']
        b = DEPARTAMENTO[i]['Destino']
        c = DEPARTAMENTO[i]['Distancia']
        d = DEPARTAMENTO[i]['CPD'] 
        midict = {'Destino':b,'Distancia':c,'CPD':d,'Latitud':DEPARTAMENTO[i]['LatitudD'],'Longitud':DEPARTAMENTO[i]['LongitudD']}
        dictionary[a].append(midict)
        Grafito = nx.Graph()

    i = -1
    vis = []
    orden = []

    for origen in dictionary.keys():
        a = origen
        orden.append(a)
        i = i + 1
        for destino_dict in dictionary[origen]:
            #for z,key in enumerate(destino_dict):
            #if key == 'CPD':
            #  codigo = key
            #else:
            b = destino_dict['Destino']
            c = destino_dict['Distancia']
            lat = destino_dict['Latitud']
            lon = destino_dict['Longitud']
            Grafito.add_edge(a, b, weight=c)
            Grafito.nodes[b]['CPD'] = destino_dict['CPD']
            Grafito.nodes[b]['Latitud'] = destino_dict['Latitud']
            Grafito.nodes[b]['Longitud'] = destino_dict['Longitud']
    

    responsePath = prim(Grafito,'ALIANZA')
    

    return json.dumps(responsePath)